In [38]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error as mse
from joblib import dump

In [2]:
df=pd.read_csv("data//clean//vehiculos.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14631 entries, 0 to 14657
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Marca                   14631 non-null  object 
 1   Modelo                  14631 non-null  object 
 2   Precio                  14631 non-null  float64
 3   Potencia                14631 non-null  float64
 4   Tipo vendedor           14631 non-null  object 
 5   Categoría               14631 non-null  object 
 6   Tipo de vehículo        14631 non-null  object 
 7   puertas                 14631 non-null  int64  
 8   Versión del país        14631 non-null  object 
 9   Garantía                14631 non-null  int64  
 10  Kilometraje             14631 non-null  float64
 11  Año                     14631 non-null  int64  
 12  Tipo de cambio          14631 non-null  object 
 13  Capacidad               14631 non-null  float64
 14  Consumo de combustible  14631 non-null  flo

Como 'Versión del país' siempre contiene el valor 'España' no es relevante y lo podemos eliminar.

In [3]:
df=df.drop('Versión del país', axis=1)

Definimos las columnas numéricas y las categóricas:

In [4]:
num_cols=df.select_dtypes(['int64', 'float64']).columns
cat_cols=df.select_dtypes(['object']).columns

De las columnas numéricas hay que mirar cuales tienen distribución gausiana, para saber a cuales podemos estandarizar con el StandartScaler.

In [5]:
def get_gaussian_columns(df):
    gaus_cols=[]
    for column in df.columns:
        stat, p = shapiro(df[column])
        if p > 0.05:
            gaus_cols.append(column)
    return gaus_cols

In [6]:
df_num=df[num_cols]
gaus_cols=get_gaussian_columns(df_num)
print(gaus_cols)

[]


c:\Users\mabi1\anaconda3\envs\proj\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Ninguna es probable que tenga distribución gausiana. Ahora de las columnas numéricas vamos a separar las que tienen outliers de las que no, para ello cogeremos el dataframe con solo las columnas numéricas, tomaremos también el primer cuantil, el tercero y la diferencia entre ambos y con una fórmula matemática seleccionaremos las columnas donde hay outliers. Una vez obtenidas las columnas con outliers haremos un drop de las mismas en el dataframe de columnas numéricas y las restantes serán las que no tienen outliers. Es importante tener en cuenta cuales tienen outliers y cuales no para saber qué tipo de escalado aplicarles.

In [7]:
q1 = df_num.quantile(0.25)
q3 = df_num.quantile(0.75)
diff = q3 - q1
out_cols=df_num.columns[((df_num < (q1 - 1.5 * diff)) |(df_num > (q3 + 1.5 * diff))).any(axis=0)]
#Conseguimos las columnas sin outliers eliminando las que tienen outliers del conjunto de columnas numéricas
no_out_cols=df_num.drop(out_cols, axis=1).columns
#Eliminamos la variable target
out_cols=out_cols.drop('Precio')

In [41]:
print('Numéricas con outliers',list(out_cols))
print('Numéricas sin outliers',list(no_out_cols))
print('Categóricas', list(cat_cols))

Numéricas con outliers ['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad', 'Consumo de combustible', 'plazas', 'Número de marchas', 'Número de cilindros', 'Peso']
Numéricas sin outliers ['Mes', 'CP']
Categóricas ['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo', 'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción', 'Tipo de combustible', 'Ciudad', 'provincia']


Así pues, ahora tenemos las columnas de nuestro dataset divididas en tres sets: columnas categóricas (cat_cols), columnas numéricas con outliers(out_cols) y columnas numéricas sin outliers (no_out_cols).

A todas las columnas le añadiremos algún tipos de SimpleImputer, no para los NaN que había, que ya se han limpiado, sino por los que podrían llegar con los nuevos datos a predecir.

A las columnas categóricas les aplicaremos OneHotEncoder y posteriormente BinaryEncoder, para poder inlcuirlas en el PCA, a las que tienen outliers RobustScaler y a las que no tienen outliers MinMaxScaler.

In [9]:
out_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'median')),
    ('scale',RobustScaler())
])

no_out_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'median')),
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer_num', SimpleImputer(strategy = 'most_frequent')),
    ('scale', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first'))
])

Añadiremos estas pequeñas pipeslines a un ColumnTransformer para que a cada tipo de columna se le aplique lo indicado. Le indicaremos remainder='drop' para que ignore cualquier columna del dataframe que no esté contemplada en esas categorías. Y n_jobs= -1 para que use todos los procesadores que tengamos en paralelo.

In [10]:
preprocessor = ColumnTransformer(transformers=[
    ('out_pipeline',out_pipeline,out_cols),
    ('no_out_pipeline',no_out_pipeline,no_out_cols),
    ('cat_pipeline', cat_pipeline, cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

Vamos a visualizar la pipeline

In [11]:
set_config(display='diagram')
display(preprocessor)

ColumnTransformer(n_jobs=-1,
                  transformers=[('out_pipeline',
                                 Pipeline(steps=[('imputer_num',
                                                  SimpleImputer(strategy='median')),
                                                 ('scale', RobustScaler())]),
                                 Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                ('no_out_pipeline',
                                 Pipeline(steps=...
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scale',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])

Separamos las variables independientes del target

In [12]:
X=df.drop('Precio', axis=1)
y=df['Precio']

Vamos a aplicarle el preprocesado a las variables independientes

In [13]:
X=preprocessor.fit_transform(X)

In [14]:
X.shape

(14631, 2929)

Al  tener casi 3000 columnas los entrenamientos de modelos llevan casi 1 hora, lo cual no es asumible si queremos probar varios modelos con varios parámetros, así que aplicaremos un PCA y nos quedaremos con el 92% de la varianza, para agilizar el proceso.

Volvemos a asignar X porque en la siguiente pipeline se volverá a preprocesar

In [18]:
X=df.drop('Precio', axis=1)

In [13]:
pca_pipe=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.94))
])
pca_pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('out_pipeline',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   RobustScaler())]),
                                                  Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                                 ('...
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scale',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.94))])

In [14]:
X=pca_pipe.fit_transform(X)
X.shape

(14631, 15)

Buscaremos los mejores modelos con los parámetros por defecto para después tunear los hiperparámetros de los mejores.

In [21]:
lr=LinearRegression()
ridge=Ridge(random_state=101)
enet=ElasticNet(random_state=101)
rfor=RandomForestRegressor(random_state=101)
gb=GradientBoostingRegressor(random_state=101)
sgd=SGDRegressor()
svr=SVR()
bridge=BayesianRidge()
kridge=KernelRidge()

In [22]:
models=[lr,ridge,enet, rfor,gb, sgd, svr, bridge, kridge]
score_means=[]
mse=[]
performance=pd.DataFrame({'model':['Linear','Ridge','ElasticNet','Random Forest','Gradient Boosting','Stochastic Gradient Descent', 'Support Vector Machines', 'Bayesian Ridge', 'Kernel Ridge']})

In [23]:
for model in models:
    score_means.append(cross_val_score(model, X, y, scoring='r2', cv=5).mean())

In [24]:
performance['r2_mean']=score_means
performance

,model,r2_mean
0,Linear,-0.491681
1,Ridge,-0.491561
2,ElasticNet,-0.229303
3,Random Forest,-0.348102
4,Gradient Boosting,-0.303463
5,Stochastic Gradient Descent,-0.453537
6,Support Vector Machines,-0.066639
7,Bayesian Ridge,-0.478335
8,Kernel Ridge,-0.636612


De momento los valores para los modelos por defecto son bastante bajos, aún así filtraremos los 4 con mejores puntuaciones para tunear hiperparámetros e intentar obtener puntuaciones mejores. 

Estos serán Support Vector Machines , ElasticNet , Gradient Boosting  y Random Forest.

In [15]:
params_SVM={
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'shrinking': [True, False]
}

In [16]:
gs_SVM = GridSearchCV(SVR(), params_SVM, cv = 5, scoring = ['r2', 'neg_mean_squared_error'], refit='r2', n_jobs = -1)
gs_SVM.fit(X, y)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'shrinking': [True, False]},
             refit='r2', scoring=['r2', 'neg_mean_squared_error'])

In [29]:
print(f'Best R2: {gs_SVM.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_SVM.best_params_}\n')

Best R2: -0.017

Best parameter set: {'kernel': 'linear', 'shrinking': True}



In [18]:
params_GB={
    'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'criterion': ['friedman_mse', 'squared_error'],
    'random_state' : [101],
    'warm_start' : [True, False]
}

In [19]:
gs_GB = GridSearchCV(GradientBoostingRegressor(), params_GB, cv = 5, scoring = ['r2', 'neg_mean_squared_error'], refit='r2', n_jobs = -1)
gs_GB.fit(X, y)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'random_state': [101], 'warm_start': [True, False]},
             refit='r2', scoring=['r2', 'neg_mean_squared_error'])

In [30]:
print(f'Best R2: {gs_GB.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_GB.best_params_}\n')

Best R2: 0.162

Best parameter set: {'criterion': 'friedman_mse', 'loss': 'absolute_error', 'random_state': 101, 'warm_start': True}



In [21]:
params_enet={
    'fit_intercept' : [True, False],
    'copy_X' : [True, False],
    'warm_start' : [True, False],
    'positive' : [True, False],
    'random_state' : [101],
    'selection' : ['cyclic', 'random']
}

In [22]:
gs_enet = GridSearchCV(ElasticNet(), params_enet, cv = 5, scoring = 'r2', n_jobs = -1)
gs_enet.fit(X, y)

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'positive': [True, False], 'random_state': [101],
                         'selection': ['cyclic', 'random'],
                         'warm_start': [True, False]},
             scoring='r2')

In [31]:
print(f'Best R2: {gs_enet.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_enet.best_params_}\n')

Best R2: 0.160

Best parameter set: {'copy_X': True, 'fit_intercept': True, 'positive': True, 'random_state': 101, 'selection': 'cyclic', 'warm_start': True}



In [42]:
params_RF={
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'oob_score' : [True, False],
    'random_state' : [101],
    'warm_start' : [True, False] 
}

In [43]:
gs_RF = GridSearchCV(RandomForestRegressor(), params_RF, cv = 5, scoring = 'r2', n_jobs = -1)
gs_RF.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'oob_score': [True, False], 'random_state': [101],
                         'warm_start': [True, False]},
             scoring='r2')

In [44]:
print(f'Best R2: {gs_RF.best_estimator_.score(X, y):.3f}\n')
print(f'Best parameter set: {gs_RF.best_params_}\n')

Best R2: 0.898

Best parameter set: {'criterion': 'poisson', 'oob_score': True, 'random_state': 101, 'warm_start': True}



Finalmente el modelo que nos ha dado mejores resultados es el Random Forest Regressor con un R2 de 0.89. También ha sido el que más ha tardado en entrenarse con diferencia. Ahora lo añadiremos a la pipeline final

In [45]:
pipe=Pipeline([
    ('preprocessing', preprocessor),
    ('pca', PCA(n_components=0.94)),
    ('model', gs_RF.best_estimator_)
])
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('out_pipeline',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scale',
                                                                   RobustScaler())]),
                                                  Index(['Potencia', 'puertas', 'Garantía', 'Kilometraje', 'Año', 'Capacidad',
       'Consumo de combustible', 'plazas', 'Número de marchas',
       'Número de cilindros', 'Peso'],
      dtype='object')),
                                                 (...
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['Marca', 'Modelo', 'Tipo vendedor', 'Categoría', 'Tipo de vehículo',
       'Tipo de cambio', 'Color exterior', 'Color original', 'Tracción',
       'Tipo de combustible', 'Ciudad', 'provincia'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.94)),
                ('model',
                 RandomForestRegressor(criterion='poisson', oob_score=True,
                                       random_state=101, warm_start=True))])

Guardaremos el dataframe y el modelo para usarlo con streamlit en el archivo 'web.py'

In [46]:
dump(df, "vehiculos.df")
dump(pipe, "model.joblib")

['model.joblib']